## Preperation: load dependencies

In [14]:
//import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.4`
//import $ivy.`com.typesafe.slick::slick:3.3.3`
//import $ivy.`com.typesafe.slick::slick-hikaricp:3.3.3`
//import $ivy.`com.github.tminglei::slick-pg:0.19.5`
//import $ivy.`com.github.tminglei::slick-pg_spray-json:0.19.5`
//import $ivy.`org.postgresql:postgresql:42.2.19`
//import $ivy.`ch.qos.logback:logback-classic:1.2.3`

// Load fat JAR
interp.load.cp(os.Path("/home/jovyan/jar/masterthesis-khipu-assembly-0.1.jar"))

In [15]:
// Smile

import $ivy.`com.github.haifengl::smile-scala:2.6.0`

import scala.language.postfixOps
import org.apache.commons.csv.CSVFormat
import smile._
import smile.util._
import smile.math._
import smile.math.MathEx._
import smile.math.kernel._
import smile.math.matrix._
import smile.math.matrix.Matrix._
import smile.math.rbf._
import smile.stat.distribution._
import smile.data._
import smile.data.formula._
import smile.data.measure._
import smile.data.`type`._
import smile.json._
import smile.interpolation._
import smile.validation._
import smile.association._
import smile.base.cart.SplitRule
import smile.base.mlp._
import smile.base.rbf.RBF
import smile.classification._
import smile.regression._
import smile.feature._
import smile.clustering._
import smile.vq._
import smile.manifold._
import smile.mds._
import smile.sequence._
import smile.projection._
import smile.nlp._
import smile.wavelet._
// Plotting
import smile.plot.swing._
import smile.plot.vega._
import smile.plot.show
import smile.plot.Render._

import $ivy.$                                       


import scala.language.postfixOps

import org.apache.commons.csv.CSVFormat

import smile._

import smile.util._

import smile.math._

import smile.math.MathEx._

import smile.math.kernel._

import smile.math.matrix._

import smile.math.matrix.Matrix._

import smile.math.rbf._

import smile.stat.distribution._

import smile.data._

import smile.data.formula._

import smile.data.measure._

import smile.data.`type`._

import smile.json._

import smile.interpolation._

import smile.validation._

import smile.association._

import smile.base.cart.SplitRule

import smile.base.mlp._

import smile.base.rbf.RBF

import smile.classification._

import smile.regression._

import smile.feature._

import smile.clustering._

import smile.vq._

import smile.manifold._

import smile.mds._

import smile.sequence._

import smile.projection._

import smile.nlp._

import smile.wavelet._
// Plotting

import smile.plot.swing._

import smile.plot.vega._

i

## Get data from DB

In [31]:
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.Future
import scala.concurrent.duration._
import scala.concurrent.{ Await, Future }
import scala.language.postfixOps

import de.tmrdlt.database.workflowlist.WorkflowListDB
import de.tmrdlt.database.action.ActionDB


val workflowListDB = new WorkflowListDB()
val actionDB = new ActionDB()

val workflowLists =  Await.result(workflowListDB.getWorkflowLists, 2 seconds)
val actions =  Await.result(actionDB.getActions, 2 seconds)

import scala.concurrent.ExecutionContext.Implicits.global

import scala.concurrent.Future

import scala.concurrent.duration._

import scala.concurrent.{ Await, Future }

import scala.language.postfixOps


import de.tmrdlt.database.workflowlist.WorkflowListDB

import de.tmrdlt.database.action.ActionDB



workflowListDB: WorkflowListDB = de.tmrdlt.database.workflowlist.WorkflowListDB@7ad4144d
actionDB: ActionDB = de.tmrdlt.database.action.ActionDB@10bab2fe
workflowLists: Seq[WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-12T13:44:20.759
  ),
  WorkflowList(
    2L,
    "5fad0555e165558a64ad0479",
    "Tasks / ToDos",
    None,
    Some(1L),
    0L,
    LIST,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-12T09:41:28.090
  ),
  WorkflowList(
    3L,
    "5fad0555e165558a64ad047a",
    "Im Gang

# Trello
## Size of boards and columns over time

In [32]:
import de.tmrdlt.models._
import de.tmrdlt.database.workflowlist._
import java.time.LocalDate
import scala.collection.mutable.ListBuffer

val boards = workflowLists
    .filter(_.listType == WorkflowListType.BOARD)
    .filter(_.dataSource == WorkflowListDataSource.Trello)
print(boards.length)

6

import de.tmrdlt.models._

import de.tmrdlt.database.workflowlist._

import java.time.LocalDate

import scala.collection.mutable.ListBuffer


boards: Seq[WorkflowList] = Vector(
  WorkflowList(
    1L,
    "5fad0555e165558a64ad0478",
    "Masterarbeit",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2020-11-12T10:50:13,
    2021-03-12T13:44:20.759
  ),
  WorkflowList(
    39L,
    "52dd0fb645357d6f614646d1",
    "Rust Art",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
    None,
    2014-01-20T12:59:50,
    2018-09-18T10:49:57.613
  ),
  WorkflowList(
    332L,
    "52ed8d9cb1f3e08864cdef28",
    "PlayMindcrack",
    Some(""),
    None,
    0L,
    BOARD,
    Some(OPEN),
    Trello,
...

In [36]:
def plotDataForBoard(board: WorkflowList) = {
    val data = new ListBuffer[(Int, LocalDate)]
    data += ((0, board.createdAt.toLocalDate))
    
    actions
        .filter(a => (a.actionType == "createCard" || a.actionType == "deleteCard") 
               && a.boardApiId == Some(board.apiId))
        .sortBy(_.date)
        .foreach { a => 
            val count = if (a.actionType == "createCard") data.last._1 + 1 else data.last._1 - 1
            data += ((count, a.date.toLocalDate))
        }
    
    val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
    val chart = VegaLite.view()
        .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
        .width(800)
        .height(200)
        .x(field = "date", `type` = "temporal")
        .y(field = "count", `type` = "quantitative")
        .data(smileData)
        .title(s"Size of the board '${board.title}' over time")
    show(chart)
    
    val columns = workflowLists
    .filter(_.listType == WorkflowListType.LIST)
    .filter(_.parentId == Some(board.id))
    .sortBy(_.position)
    val columnData = 
    columns.map { column =>
        val data = new ListBuffer[(Int, LocalDate, String, Int)]
        data += ((0, column.createdAt.toLocalDate, column.title, column.position.toInt))
        
        actions
            .filter(a => a.parentApiId == Some(column.apiId) ||
                         a.oldParentApiId == Some(column.apiId) ||
                         a.newParentApiId == Some(column.apiId))
            .sortBy(_.date)
            .foreach { a => 
                val count = 
                    if (a.actionType == "createCard") {
                        data.last._1 + 1
                    } else if (a.actionType == "deleteCard") {
                        data.last._1 - 1  
                    } else {
                        if (column.apiId == a.oldParentApiId.get) {
                            data.last._1 - 1
                        } else {
                            data.last._1 + 1
                        }
                    } 
                data += ((count, a.date.toLocalDate, column.title, column.position.toInt))
            }
        data
    }
    
    columnData.foreach { data =>
        val smileData = data.toList.map(d => JsObject("date" -> JsDate(d._2), "count" -> JsInt(d._1)))
        val chart = VegaLite.view()
            .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
            .width(800)
            .height(200)
            .x(field = "date", `type` = "temporal")
            .y(field = "count", `type` = "quantitative")
            .data(smileData)
            .title(s"Size of column '${data(0)._3}' over time")
        show(chart)
    }
    // Plot as Facet
    //val columnSmileData = columnData.flatten.toList.map(d => JsObject("card_count" -> JsInt(d._1), "date" -> JsDate(d._2), "column_name" -> JsString(d._3), "position" -> JsInt(d._4)))
    //val chart2 = VegaLite.facet(columnSmileData)
    //    .row(field = "column_name", `type` = "nominal")
    //    .mark(JsObject("type" -> "area", "line" -> true, "point" -> true))
    //    .width(800)
    //    .height(150)
    //    .x(field = "date", `type` = "temporal")
    //    .y(field = "card_count", `type` = "quantitative")
    //    .title(s"Size of the columns of board '${board.title}' over time")
    //show(chart2)
}

defined function plotDataForBoard

In [37]:
plotDataForBoard(boards(4))

<iframe id="e93688a3-1a23-4a44-b60d-cc1785670bc9" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2013-09-28","count":0},{"date":"2013-09-27","count":1},{"date":"2013-09-30","count":2},{"date":"2013-10-02","count":3},{"date":"2013-10-04","count":4},{"date":"2013-10-08","count":5},{"date":"2013-10-09","count":6},{"date":"2013-10-15","count":7},{"date":"2013-10-15","count":8},{"date":"2013-10-23","count":9},{"date":"2013-10-23","count":10},{"date":"2013-10-24","count":11},{"date":"2013-10-25","count":12},{"date":"2013-10-25","count":13},{"date":"2013-10-29","count":14},{"date":"2013-11-05","count":15},{"date":"2013-11-11","count":16},{"date":"2013-11-12","count":17},{"date":"2013-11-13","count":18},{"date":"2013-11-15","count":19},{"date":"2013-11-16","count":20},{"date":"2013-11-27","count":21},{"date":"2013-12-02","count":22},{"date":"2013-12-04","count":23},{"date":"2013-12-17","count":24},{"date":"2013-12-18","count":25},{"date":"2014-01-07","count":26},{"date":"2014-01-21","count":27},{"date":"2014-01-22","count":28},{"date":"2014-01-22","count":29},{"date":"2014-01-25","count":30},{"date":"2014-01-30","count":31},{"date":"2014-01-30","count":32},{"date":"2014-02-26","count":33},{"date":"2014-02-27","count":34},{"date":"2014-03-03","count":35},{"date":"2014-03-05","count":36},{"date":"2014-03-05","count":37},{"date":"2014-03-08","count":38},{"date":"2014-03-10","count":39},{"date":"2014-03-11","count":40},{"date":"2014-03-11","count":41},{"date":"2014-03-11","count":42},{"date":"2014-03-18","count":43},{"date":"2014-03-24","count":44},{"date":"2014-03-24","count":45},{"date":"2014-03-24","count":46},{"date":"2014-03-24","count":47},{"date":"2014-03-24","count":48},{"date":"2014-03-28","count":49},{"date":"2014-03-28","count":50},{"date":"2014-04-01","count":51},{"date":"2014-04-02","count":52},{"date":"2014-04-02","count":53},{"date":"2014-04-02","count":54},{"date":"2014-04-03","count":55},{"date":"2014-04-03","count":56},{"date":"2014-04-03","count":57},{"date":"2014-04-03","count":58},{"date":"2014-04-03","count":59},{"date":"2014-04-03","count":60},{"date":"2014-04-03","count":61},{"date":"2014-04-03","count":62},{"date":"2014-04-03","count":63},{"date":"2014-04-04","count":64},{"date":"2014-04-04","count":65},{"date":"2014-04-04","count":66},{"date":"2014-04-08","count":67},{"date":"2014-04-08","count":68},{"date":"2014-04-09","count":69},{"date":"2014-04-09","count":70},{"date":"2014-04-09","count":71},{"date":"2014-04-09","count":72},{"date":"2014-04-09","count":73},{"date":"2014-04-09","count":74},{"date":"2014-04-15","count":75},{"date":"2014-04-15","count":76},{"date":"2014-04-15","count":77},{"date":"2014-04-15","count":78},{"date":"2014-04-15","count":79},{"date":"2014-04-16","count":80},{"date":"2014-04-18","count":81},{"date":"2014-04-21","count":82},{"date":"2014-04-22","count":83},{"date":"2014-04-22","count":84},{"date":"2014-04-23","count":85},{"date":"2014-04-23","count":86},{"date":"2014-04-23","count":87},{"date":"2014-04-23","count":88},{"date":"2014-04-24","count":89},{"date":"2014-04-28","count":90},{"date":"2014-04-28","count":91},{"date":"2014-04-28","count":92},{"date":"2014-04-28","count":93},{"date":"2014-04-30","count":94},{"date":"2014-04-30","count":95},{"date":"2014-04-30","count":96},{"date":"2014-04-30","count":97},{"date":"2014-04-30","count":98},{"date":"2014-05-01","count":99},{"date":"2014-05-01","count":100},{"date":"2014-05-01","count":101},{"date":"2014-05-05","count":102},{"date":"2014-05-05","count":103},{"date":"2014-05-05","count":104},{"dat

<iframe id="beba8ef2-f0af-45d1-9452-256ac07d7be4" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2015-02-27","count":0},{"date":"2015-02-27","count":1},{"date":"2015-02-27","count":2},{"date":"2015-02-27","count":3},{"date":"2015-02-27","count":4},{"date":"2015-02-27","count":5},{"date":"2015-02-27","count":6},{"date":"2015-02-27","count":7},{"date":"2015-02-27","count":8},{"date":"2015-02-27","count":9},{"date":"2015-02-27","count":10},{"date":"2015-02-27","count":11},{"date":"2015-02-27","count":12},{"date":"2015-02-27","count":13},{"date":"2015-02-27","count":14},{"date":"2015-02-27","count":15},{"date":"2015-02-27","count":16},{"date":"2015-02-27","count":17},{"date":"2015-02-27","count":18},{"date":"2015-02-27","count":19},{"date":"2015-02-27","count":20},{"date":"2015-02-27","count":21},{"date":"2015-02-27","count":22},{"date":"2015-02-27","count":23},{"date":"2015-02-27","count":24},{"date":"2015-02-27","count":25},{"date":"2015-02-27","count":26},{"date":"2015-02-27","count":27},{"date":"2015-02-27","count":28},{"date":"2015-02-27","count":29},{"date":"2015-02-27","count":30},{"date":"2015-02-27","count":31},{"date":"2015-02-27","count":32},{"date":"2015-02-27","count":33},{"date":"2015-02-27","count":34},{"date":"2015-02-27","count":35},{"date":"2015-02-27","count":36},{"date":"2015-02-27","count":37},{"date":"2015-02-27","count":38},{"date":"2015-02-27","count":39},{"date":"2015-02-27","count":40},{"date":"2015-02-27","count":41},{"date":"2015-02-27","count":42},{"date":"2015-02-27","count":43},{"date":"2015-02-27","count":44},{"date":"2015-02-27","count":45},{"date":"2015-02-27","count":46},{"date":"2015-02-27","count":47},{"date":"2015-02-27","count":48},{"date":"2015-02-27","count":49},{"date":"2015-02-27","count":50},{"date":"2015-02-27","count":51},{"date":"2015-02-27","count":52},{"date":"2015-02-27","count":53},{"date":"2015-02-27","count":54},{"date":"2015-02-27","count":55},{"date":"2015-02-27","count":56},{"date":"2015-02-27","count":57},{"date":"2015-02-27","count":58},{"date":"2015-02-27","count":59},{"date":"2015-02-27","count":60},{"date":"2015-02-27","count":61},{"date":"2015-02-27","count":62},{"date":"2015-02-27","count":63},{"date":"2015-02-27","count":64},{"date":"2015-02-27","count":65},{"date":"2015-02-27","count":66},{"date":"2015-02-27","count":67},{"date":"2015-02-27","count":68},{"date":"2015-02-27","count":69},{"date":"2015-02-27","count":70},{"date":"2015-02-27","count":71},{"date":"2015-02-27","count":72},{"date":"2015-02-27","count":73},{"date":"2015-02-27","count":74},{"date":"2015-02-27","count":75},{"date":"2015-02-27","count":76},{"date":"2015-02-27","count":77},{"date":"2015-02-27","count":78},{"date":"2015-02-27","count":79},{"date":"2015-02-27","count":80},{"date":"2015-02-27","count":81},{"date":"2015-02-27","count":82},{"date":"2015-02-27","count":83},{"date":"2015-02-27","count":84},{"date":"2015-02-27","count":85},{"date":"2015-02-27","count":86},{"date":"2015-02-27","count":87},{"date":"2015-02-27","count":88},{"date":"2015-02-27","count":89},{"date":"2015-02-27","count":90},{"date":"2015-02-27","count":91},{"date":"2015-02-27","count":92},{"date":"2015-02-27","count":93},{"date":"2015-02-27","count":94},{"date":"2015-02-27","count":95},{"date":"2015-02-27","count":96},{"date":"2015-02-27","count":97},{"date":"2015-02-27","count":98},{"date":"2015-02-27","count":99},{"date":"2015-02-27","count":100},{"date":"2015-02-27","count":101},{"date":"2015-02-27","count":102},{"date":"2015-02-27","count":103},{"date":"2015-02-27","count":104},{"dat

<iframe id="4bc2d920-9015-4d5d-a128-2c47a534e017" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2015-09-11","count":0},{"date":"2015-09-11","count":1},{"date":"2015-09-11","count":2},{"date":"2015-09-11","count":3},{"date":"2015-09-11","count":2},{"date":"2015-09-11","count":3},{"date":"2015-09-11","count":4},{"date":"2015-09-11","count":5},{"date":"2015-09-11","count":6},{"date":"2015-09-14","count":5},{"date":"2015-09-14","count":4},{"date":"2015-09-14","count":3},{"date":"2015-09-14","count":4},{"date":"2015-09-14","count":3},{"date":"2015-09-14","count":4},{"date":"2015-09-14","count":3},{"date":"2015-09-14","count":4},{"date":"2015-09-15","count":5},{"date":"2015-09-15","count":4},{"date":"2015-09-15","count":3},{"date":"2015-09-16","count":4},{"date":"2015-09-17","count":5},{"date":"2015-09-17","count":6},{"date":"2015-09-18","count":5},{"date":"2015-09-18","count":4},{"date":"2015-09-18","count":3},{"date":"2015-09-18","count":4},{"date":"2015-09-21","count":3},{"date":"2015-09-21","count":2},{"date":"2015-09-21","count":1},{"date":"2015-09-22","count":2},{"date":"2015-09-22","count":1},{"date":"2015-09-22","count":0},{"date":"2015-09-22","count":-1},{"date":"2015-09-23","count":-2},{"date":"2015-09-23","count":-3},{"date":"2015-09-23","count":-4},{"date":"2015-09-23","count":-5},{"date":"2015-09-24","count":-6},{"date":"2015-09-24","count":-7},{"date":"2015-09-24","count":-8},{"date":"2015-09-24","count":-7},{"date":"2015-09-24","count":-8},{"date":"2015-09-24","count":-7},{"date":"2015-09-24","count":-6},{"date":"2015-09-25","count":-7},{"date":"2015-09-28","count":-6},{"date":"2015-09-28","count":-5},{"date":"2015-09-28","count":-4},{"date":"2015-09-28","count":-5},{"date":"2015-09-28","count":-4},{"date":"2015-09-28","count":-3},{"date":"2015-09-29","count":-2},{"date":"2015-09-29","count":-1},{"date":"2015-09-30","count":0},{"date":"2015-09-30","count":1},{"date":"2015-10-01","count":0},{"date":"2015-10-01","count":-1},{"date":"2015-10-02","count":-2},{"date":"2015-10-02","count":-1},{"date":"2015-10-02","count":0},{"date":"2015-10-03","count":-1},{"date":"2015-10-05","count":0},{"date":"2015-10-05","count":-1},{"date":"2015-10-05","count":-2},{"date":"2015-10-05","count":-3},{"date":"2015-10-07","count":-4},{"date":"2015-10-07","count":-5},{"date":"2015-10-09","count":-6},{"date":"2015-10-10","count":-7},{"date":"2015-10-12","count":-8},{"date":"2015-10-12","count":-9},{"date":"2015-10-12","count":-10},{"date":"2015-10-12","count":-9},{"date":"2015-10-13","count":-10},{"date":"2015-10-13","count":-11},{"date":"2015-10-13","count":-12},{"date":"2015-10-14","count":-13},{"date":"2015-10-14","count":-12},{"date":"2015-10-15","count":-13},{"date":"2015-10-19","count":-14},{"date":"2015-10-19","count":-15},{"date":"2015-10-19","count":-16},{"date":"2015-10-19","count":-15},{"date":"2015-10-19","count":-14},{"date":"2015-10-19","count":-15},{"date":"2015-10-19","count":-14},{"date":"2015-10-20","count":-15},{"date":"2015-10-20","count":-16},{"date":"2015-10-21","count":-15},{"date":"2015-10-21","count":-14},{"date":"2015-10-22","count":-15},{"date":"2015-10-23","count":-16},{"date":"2015-10-23","count":-15},{"date":"2015-10-24","count":-16},{"date":"2015-10-24","count":-17},{"date":"2015-10-24","count":-18},{"date":"2015-10-24","count":-19},{"date":"2015-10-27","count":-20},{"date":"2015-10-27","count":-21},{"date":"2015-10-27","count":-20},{"date":"2015-10-28","count":-21},{"date":"2015-10-28","count":-22},{"date":"2015-10-29","count":-23},{"date":"2015-10-29","count":-24},{"date

<iframe id="acebed0f-28e4-4225-bdb8-c0dc5a1edf1c" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2015-09-11","count":0},{"date":"2015-09-11","count":1},{"date":"2015-09-11","count":2},{"date":"2015-09-11","count":3},{"date":"2015-09-11","count":4},{"date":"2015-09-11","count":5},{"date":"2015-09-11","count":6},{"date":"2015-09-11","count":7},{"date":"2015-09-11","count":6},{"date":"2015-09-11","count":5},{"date":"2015-09-11","count":4},{"date":"2015-09-14","count":5},{"date":"2015-09-14","count":6},{"date":"2015-09-14","count":7},{"date":"2015-09-14","count":8},{"date":"2015-09-14","count":7},{"date":"2015-09-15","count":6},{"date":"2015-09-15","count":7},{"date":"2015-09-15","count":6},{"date":"2015-09-17","count":5},{"date":"2015-09-18","count":4},{"date":"2015-09-18","count":5},{"date":"2015-09-18","count":6},{"date":"2015-09-18","count":7},{"date":"2015-09-18","count":6},{"date":"2015-09-18","count":7},{"date":"2015-09-18","count":8},{"date":"2015-09-18","count":7},{"date":"2015-09-18","count":6},{"date":"2015-09-18","count":5},{"date":"2015-09-21","count":6},{"date":"2015-09-21","count":7},{"date":"2015-09-21","count":6},{"date":"2015-09-21","count":7},{"date":"2015-09-21","count":6},{"date":"2015-09-21","count":7},{"date":"2015-09-22","count":6},{"date":"2015-09-22","count":5},{"date":"2015-09-22","count":6},{"date":"2015-09-22","count":7},{"date":"2015-09-22","count":8},{"date":"2015-09-22","count":7},{"date":"2015-09-23","count":6},{"date":"2015-09-23","count":7},{"date":"2015-09-23","count":8},{"date":"2015-09-23","count":7},{"date":"2015-09-23","count":8},{"date":"2015-09-23","count":9},{"date":"2015-09-23","count":10},{"date":"2015-09-24","count":9},{"date":"2015-09-24","count":10},{"date":"2015-09-24","count":9},{"date":"2015-09-24","count":8},{"date":"2015-09-24","count":7},{"date":"2015-09-24","count":6},{"date":"2015-09-25","count":5},{"date":"2015-09-25","count":4},{"date":"2015-09-25","count":5},{"date":"2015-09-25","count":4},{"date":"2015-09-28","count":5},{"date":"2015-09-28","count":6},{"date":"2015-09-28","count":5},{"date":"2015-09-28","count":4},{"date":"2015-09-28","count":3},{"date":"2015-09-28","count":4},{"date":"2015-09-29","count":3},{"date":"2015-09-29","count":2},{"date":"2015-09-30","count":1},{"date":"2015-10-02","count":0},{"date":"2015-10-02","count":1},{"date":"2015-10-03","count":2},{"date":"2015-10-03","count":1},{"date":"2015-10-05","count":2},{"date":"2015-10-07","count":3},{"date":"2015-10-07","count":4},{"date":"2015-10-09","count":5},{"date":"2015-10-10","count":6},{"date":"2015-10-12","count":5},{"date":"2015-10-12","count":4},{"date":"2015-10-12","count":3},{"date":"2015-10-12","count":2},{"date":"2015-10-12","count":1},{"date":"2015-10-12","count":0},{"date":"2015-10-12","count":-1},{"date":"2015-10-12","count":-2},{"date":"2015-10-12","count":-1},{"date":"2015-10-12","count":-2},{"date":"2015-10-12","count":-1},{"date":"2015-10-12","count":-2},{"date":"2015-10-13","count":-1},{"date":"2015-10-13","count":0},{"date":"2015-10-13","count":-1},{"date":"2015-10-13","count":-2},{"date":"2015-10-13","count":-1},{"date":"2015-10-13","count":-2},{"date":"2015-10-13","count":-1},{"date":"2015-10-13","count":-2},{"date":"2015-10-13","count":-3},{"date":"2015-10-13","count":-4},{"date":"2015-10-14","count":-3},{"date":"2015-10-14","count":-4},{"date":"2015-10-14","count":-5},{"date":"2015-10-14","count":-6},{"date":"2015-10-14","count":-7},{"date":"2015-10-15","count":-8},{"date":"2015-10-15","count":-7},{"date":"2015-10-19","count":-6},{"date":"2015-10

<iframe id="2068cbff-61ed-477e-a7d3-e379b9fb3814" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2019-03-31","count":0},{"date":"2019-05-03","count":1},{"date":"2019-05-03","count":2},{"date":"2019-05-03","count":3},{"date":"2019-05-03","count":2}]},"width":800,"title":"Size of column 'Done B85: Maintenance' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="75942cc7-3f2d-4084-8bbe-85518d623122" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2018-02-06","count":0},{"date":"2018-02-13","count":1},{"date":"2018-02-13","count":0},{"date":"2018-03-02","count":1},{"date":"2018-03-21","count":2},{"date":"2018-04-24","count":3},{"date":"2018-05-10","count":4},{"date":"2018-07-02","count":5},{"date":"2018-09-12","count":6},{"date":"2018-09-13","count":7},{"date":"2018-09-13","count":8},{"date":"2018-09-13","count":9},{"date":"2018-09-13","count":10},{"date":"2018-09-27","count":11},{"date":"2018-09-27","count":12},{"date":"2018-09-27","count":13},{"date":"2018-10-04","count":14},{"date":"2018-10-08","count":15},{"date":"2018-10-09","count":16},{"date":"2018-10-10","count":17}]},"width":800,"title":"Size of column 'Done B84: Maintenance' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="2c603994-1feb-4adb-989c-22c7cf53e7b9" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2018-01-11","count":0},{"date":"2018-01-11","count":1},{"date":"2018-01-11","count":2},{"date":"2018-01-12","count":3},{"date":"2018-01-12","count":4},{"date":"2018-01-12","count":5},{"date":"2018-01-15","count":6},{"date":"2018-01-15","count":7},{"date":"2018-01-16","count":8},{"date":"2018-01-16","count":9},{"date":"2018-01-16","count":10},{"date":"2018-01-16","count":11},{"date":"2018-01-16","count":12},{"date":"2018-01-19","count":13},{"date":"2018-01-27","count":14},{"date":"2018-02-06","count":15},{"date":"2018-02-06","count":16},{"date":"2018-02-13","count":17},{"date":"2018-02-13","count":18},{"date":"2018-02-13","count":17},{"date":"2018-02-13","count":18}]},"width":800,"title":"Size of column 'Done B83: Bugfixes' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="08edc855-3327-4018-a4a6-78c657e91713" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2017-12-09","count":0},{"date":"2017-12-09","count":1},{"date":"2017-12-09","count":2},{"date":"2017-12-11","count":3},{"date":"2017-12-11","count":2},{"date":"2017-12-11","count":3},{"date":"2017-12-11","count":4},{"date":"2017-12-11","count":5},{"date":"2017-12-11","count":6},{"date":"2017-12-11","count":7},{"date":"2017-12-11","count":8},{"date":"2017-12-12","count":9},{"date":"2017-12-15","count":10},{"date":"2017-12-15","count":11},{"date":"2017-12-19","count":12},{"date":"2017-12-19","count":13},{"date":"2017-12-21","count":14},{"date":"2017-12-21","count":15},{"date":"2017-12-21","count":16},{"date":"2017-12-21","count":17},{"date":"2018-01-01","count":18},{"date":"2018-01-02","count":19},{"date":"2018-01-02","count":18},{"date":"2018-01-02","count":19},{"date":"2018-01-02","count":20},{"date":"2018-01-02","count":21},{"date":"2018-01-02","count":22},{"date":"2018-01-02","count":23},{"date":"2018-01-03","count":24},{"date":"2018-01-03","count":25},{"date":"2018-01-04","count":26},{"date":"2018-01-04","count":27},{"date":"2018-01-04","count":28},{"date":"2018-01-04","count":29},{"date":"2018-01-04","count":30},{"date":"2018-01-04","count":31},{"date":"2018-01-04","count":32},{"date":"2018-01-04","count":33},{"date":"2018-01-05","count":34},{"date":"2018-01-05","count":35},{"date":"2018-01-06","count":36},{"date":"2018-01-08","count":37},{"date":"2018-01-08","count":38},{"date":"2018-01-08","count":39},{"date":"2018-01-09","count":40},{"date":"2018-01-09","count":41},{"date":"2018-01-09","count":42},{"date":"2018-01-09","count":43},{"date":"2018-01-09","count":42},{"date":"2018-01-09","count":43},{"date":"2018-01-10","count":44},{"date":"2018-01-10","count":45},{"date":"2018-01-10","count":46},{"date":"2018-01-10","count":47},{"date":"2018-01-10","count":48},{"date":"2018-01-10","count":49},{"date":"2018-01-10","count":50},{"date":"2018-01-10","count":51},{"date":"2018-01-10","count":52},{"date":"2018-01-10","count":53},{"date":"2018-01-10","count":54},{"date":"2018-01-10","count":55},{"date":"2018-01-11","count":54},{"date":"2018-01-11","count":53},{"date":"2018-01-11","count":52},{"date":"2018-01-11","count":51},{"date":"2018-01-11","count":50},{"date":"2018-01-11","count":51},{"date":"2018-01-11","count":52},{"date":"2018-01-11","count":51},{"date":"2018-01-11","count":50},{"date":"2018-01-13","count":51},{"date":"2018-02-10","count":52}]},"width":800,"title":"Size of column 'Done B82: Bugfixes' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">

<iframe id="ca33380e-e7a5-4c6e-ac44-c939fbc2e772" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="
<!DOCTYPE html>
<html>
<head>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega@5"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
 <script type="text/javascript" src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
</head>
<body>

<div id="vega-lite"></div>

<script type="text/javascript">
 var spec = {"data":{"values":[{"date":"2017-10-30","count":0},{"date":"2017-10-30","count":1},{"date":"2017-10-30","count":2},{"date":"2017-10-30","count":3},{"date":"2017-10-31","count":4},{"date":"2017-11-01","count":5},{"date":"2017-11-01","count":6},{"date":"2017-11-01","count":7},{"date":"2017-11-01","count":8},{"date":"2017-11-01","count":9},{"date":"2017-11-02","count":10},{"date":"2017-11-02","count":11},{"date":"2017-11-03","count":12},{"date":"2017-11-03","count":13},{"date":"2017-11-06","count":14},{"date":"2017-11-06","count":15},{"date":"2017-11-07","count":16},{"date":"2017-11-08","count":17},{"date":"2017-11-09","count":18},{"date":"2017-11-09","count":19},{"date":"2017-11-09","count":20},{"date":"2017-11-09","count":21},{"date":"2017-11-10","count":20},{"date":"2017-11-13","count":21},{"date":"2017-11-13","count":22},{"date":"2017-11-14","count":23},{"date":"2017-11-14","count":24},{"date":"2017-11-15","count":25},{"date":"2017-11-15","count":26},{"date":"2017-11-16","count":27},{"date":"2017-11-17","count":28},{"date":"2017-11-17","count":29},{"date":"2017-11-17","count":30},{"date":"2017-11-17","count":31},{"date":"2017-11-17","count":32},{"date":"2017-11-17","count":33},{"date":"2017-11-18","count":34},{"date":"2017-11-20","count":35},{"date":"2017-11-20","count":36},{"date":"2017-11-21","count":37},{"date":"2017-11-22","count":38},{"date":"2017-11-22","count":39},{"date":"2017-11-24","count":40},{"date":"2017-11-24","count":41},{"date":"2017-11-27","count":42},{"date":"2017-11-28","count":43},{"date":"2017-11-29","count":44},{"date":"2017-11-30","count":45},{"date":"2017-12-03","count":46},{"date":"2017-12-04","count":47},{"date":"2017-12-04","count":48},{"date":"2017-12-04","count":49},{"date":"2017-12-05","count":50},{"date":"2017-12-05","count":51},{"date":"2017-12-06","count":52},{"date":"2017-12-06","count":53},{"date":"2017-12-06","count":54},{"date":"2017-12-07","count":55},{"date":"2017-12-09","count":56},{"date":"2017-12-09","count":55},{"date":"2017-12-11","count":56},{"date":"2017-12-18","count":57},{"date":"2017-12-21","count":58},{"date":"2018-01-02","count":59},{"date":"2018-01-02","count":58},{"date":"2018-01-02","count":59},{"date":"2018-01-02","count":60},{"date":"2018-01-02","count":61},{"date":"2018-01-02","count":60},{"date":"2018-01-03","count":61},{"date":"2018-01-07","count":62},{"date":"2018-01-10","count":63},{"date":"2018-01-11","count":64},{"date":"2018-01-11","count":65},{"date":"2018-01-11","count":66},{"date":"2018-01-11","count":67},{"date":"2018-01-11","count":68},{"date":"2018-01-11","count":69},{"date":"2018-01-13","count":70},{"date":"2019-03-31","count":71}]},"width":800,"title":"Size of column 'Done B81: Bugfixes' over time","encoding":{"x":{"field":"date","type":"temporal"},"y":{"field":"count","type":"quantitative"}},"mark":{"type":"area","line":true,"point":true},"height":200};
 var opt = {
 "mode": "vega-lite",
 "renderer": "canvas",
 "actions": {"editor": true, "source": true, "export": true}
 };
 vegaEmbed('#vega-lite', spec, opt).catch(console.error);
</script>
</body>
</html>
 ">